# Problem 1 

## (a)

\begin{align*}

& dC = \frac{\partial C(S_t, t)}{\partial t} dt + \frac{\partial C(S_t, t )}{\partial S_t,t} dS_t + \frac{1}{2} \frac{\partial^2 C(S_t,t)}{\partial S_t^2 } d<S_t> \\
& dS_t = \sigma S_t^{1+\alpha} dW_t \\
& dC = \frac{\partial C(S_t, t)}{\partial t} dt + \frac{\partial C(S_t, t )}{\partial S_t,t} \sigma S_t^{1+\alpha} dW_t  + \frac{1}{2} \frac{\partial^2 C(S_t,t)}{\partial S_t^2 } \sigma^2 S_t^{2(1+\alpha)} dt \\
& dC = (\frac{\partial C(S_t, t)}{\partial t} +  \frac{1}{2} \frac{\partial^2 C(S_t,t)}{\partial S_t^2 } \sigma^2 S_t^{2(1+\alpha)} ) dt + (\frac{\partial C(S_t, t )}{\partial S_t,t} \sigma S_t^{1+\alpha}) dW_t \\
& \text{Since we know the drift term is rC, then we can simplify the equation:} \\
&  \frac{\partial C }{\partial t} +  \frac{1}{2}\sigma^2 S_t^{2(1+\alpha)} \frac{\partial^2 C }{\partial S_t^2 }   = rC , \text{ with the terminal condiction: } C_T = (K - S_T)^+ \\

\end{align*}

## (b)

In [1]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

In [2]:
class CEV:

    def __init__(self,volcoeff,alpha,rGrow,r,S0):
        self.volcoeff = volcoeff
        self.alpha = alpha
        self.rGrow = rGrow
        self.r = r
        self.S0 = S0


In [3]:
hw4dynamics = CEV(volcoeff=3, alpha=-0.5, rGrow=0, r=0.05, S0=100)

In [4]:
class Put:

    def __init__(self,T,K):
        self.T = T;
        self.K = K;

In [5]:
hw4contract=Put(T=0.25,K=100)

$\frac{\partial C }{\partial t} +  \frac{1}{2}\sigma^2 S_t^{2(1+\alpha)} \frac{\partial^2 C }{\partial S_t^2 }   = rC$

$\frac{\partial C }{\partial t} +  f(x,t) \frac{\partial^2 C }{\partial S_t^2 } + g(x,t)\frac{\partial C}{\partial S_t} + h(x,t)C  = 0$

Since we know that S follows a CEV model, then we know that $dS_t = \mu S_t dt + \sigma S_t^{1+\alpha}dW_t$

$h = - r, f =  \frac{1}{2}\sigma^2 S_t^{2(1+\alpha)}, g = \mu S_t$

In [14]:
class FD_CrankNicolson_Engine:

    def __init__(self,SMax,SMin,deltaS,deltat):
        self.SMax=SMax
        self.SMin=SMin
        self.deltaS=deltaS
        self.deltat=deltat



    def price_put_CEV(self,contract,dynamics):

        alpha, r, rGrow, volcoeff = dynamics.alpha, dynamics.r, dynamics.rGrow, dynamics.volcoeff


        N=round(contract.T/self.deltat)
        if abs(N-contract.T/self.deltat)>1e-12:
            raise ValueError('Bad time step')
        numS=round((self.SMax-self.SMin)/self.deltaS)+1
        if abs(numS-(self.SMax-self.SMin)/self.deltaS-1)>1e-12:
            raise ValueError('Bad time step')
        S=np.linspace(self.SMax,self.SMin,numS)  
        S_lowboundary=self.SMin-self.deltaS
        
        putprice=np.maximum(contract.K-S,0)

        ratio1 = self.deltat/self.deltaS
        ratio2 = self.deltat/self.deltaS**2
        f = 0.5* (volcoeff ** 2) * (S ** (2 + 2* alpha))   
        g = rGrow * S   
        h = -r  
        F = 0.5*ratio2*f + 0.25*ratio1*g
        G =     ratio2*f - 0.50*self.deltat*h
        H = 0.5*ratio2*f - 0.25*ratio1*g

        #Right-hand-side matrix
        RHSmatrix = diags([H[:-1], 1-G, F[1:]], [1,0,-1], shape=(numS,numS), format="csr")

        #Left-hand-side matrix
        LHSmatrix = diags([-H[:-1], 1+G, -F[1:]], [1,0,-1], shape=(numS,numS), format="csr")


        for t in np.arange(N-1,-1,-1)*self.deltat:

            rhs = RHSmatrix * putprice


            rhs[-1]=rhs[-1]+2*H[-1]*(contract.K-S_lowboundary)

            putprice = spsolve(LHSmatrix, rhs) 
            putprice = np.maximum(putprice, contract.K-S)

        return(S, putprice)


In [15]:
hw4FD = FD_CrankNicolson_Engine(SMax=200,SMin=50,deltaS=0.1,deltat=0.0005)

In [16]:
(S0_all, putprice) = hw4FD.price_put_CEV(hw4contract,hw4dynamics)

In [17]:

displayStart = hw4dynamics.S0-hw4FD.deltaS*1.5
displayEnd   = hw4dynamics.S0+hw4FD.deltaS*1.5
displayrows  = (S0_all>displayStart) & (S0_all<displayEnd)
np.set_printoptions(precision=4, suppress=True)
print(np.stack((S0_all, putprice),axis=1)[displayrows])

[[100.1      5.8704]
 [100.       5.9183]
 [ 99.9      5.9665]]


## (c)

\begin{align*}
& \Delta \approx \frac{C(S+\Delta S,t) - C(S-\Delta S,t)}{2\Delta S} = ( 5.8704 - 5.9665 )/(2*0.1) = -0.4805 \\
& \Gamma \approx \frac{C(S+\Delta S,t)- 2C(S,t) + C(S-\Delta S,t)}{(\Delta S)^2} = ( 5.8704 - 2* 5.9183 +5.9665 )/(0.1^2) \approx 0.0300
\end{align*}

## (d)

In [21]:
hw4contract_d=Put(T=0.25,K=100)
hw4dynamics_d = CEV(volcoeff=0.3, alpha=0, rGrow=0.05, r=0.05, S0=100)

In [22]:
hw4FD_d = FD_CrankNicolson_Engine(SMax=200,SMin=50,deltaS=0.1,deltat=0.0005)
(S0_all_d, putprice_d) = hw4FD_d.price_put_CEV(hw4contract_d,hw4dynamics_d)

In [23]:
displayStart = hw4dynamics_d.S0-hw4FD_d.deltaS*1.5
displayEnd   = hw4dynamics_d.S0+hw4FD_d.deltaS*1.5
displayrows  = (S0_all_d>displayStart) & (S0_all_d<displayEnd)
np.set_printoptions(precision=4, suppress=True)
print(np.stack((S0_all_d, putprice_d),axis=1)[displayrows])

[[100.1      5.3973]
 [100.       5.442 ]
 [ 99.9      5.487 ]]


# Problem 2

## (a)

\begin{align*}
& P(win) = \frac{34}{44}; P(lose) = \frac{10}{34}\\
& E[X] = 1 * \frac{34}{44} + 0 * \frac{10}{44} = \frac{34}{44} = \frac{17}{22}  \\
& std[X] = \sqrt{\frac{34}{44}*\frac{10}{44}} = \frac{\sqrt{85}}{22} \\
\end{align*}

## (b)

\begin{align*}
& p = \frac{17}{22}; q = \frac{5}{22} \\
& E[X] = E[R_1]+E[R_2]+E[R_3] = 3E[R_i] = (\frac{17}{22} * \frac{1}{3} + \frac{5}{22} * 0)*3 = \frac{17}{22} \\
& Var[X] = 3Var[X_i] = 3*(E[R_i^2]-E[R_i]^2) = 3 (\frac{1}{9}\frac{17}{22} - (\frac{1}{3}\frac{17}{22})^2) = \frac{85}{1452} \\
& std[X] = \sqrt{\frac{85}{1452}} \approx 0.24195
\end{align*}

## (c)

\begin{align*}
& P(x= 0) = \frac{\binom{10}{3}}{\binom{44}{3}} = \frac{120}{13244} \approx 0.00906, \\
& P(x= \frac{1}{3}) = \frac{\binom{34}{1} \binom{10}{2}}{\binom{44}{3}} = \frac{34 \cdot 45}{13244} \approx 0.11552, \\
& P(x= \frac{2}{3}) = \frac{\binom{34}{2} \binom{10}{1}}{\binom{44}{3}} = \frac{561 \cdot 10}{13244} \approx 0.42358, \\
& P(x= 1) = \frac{\binom{34}{3}}{\binom{44}{3}} = \frac{5984}{13244} \approx 0.45182, \\

& E[X] = \frac{1}{3} \cdot 0.11552 + \frac{2}{3} \cdot 0.42358 + 1 \cdot 0.45182 \approx 0.7727, \\
& E[X^2] = \left(\frac{1}{3}\right)^2 \cdot 0.11552 + \left(\frac{2}{3}\right)^2 \cdot 0.42358 + 1^2 \cdot 0.45182 \approx 0.6529, \\
& \text{Std}[X] = \sqrt{0.6529 - 0.7727^2} \approx 0.23629. \\
\end{align*}


The standard deviation is smaller than the standard deviation in b. It make sense since the probability for each round now is not the same for c. 